In [2]:
%load_ext autoreload
%autoreload 2
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.lfda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance

from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, material = read_VSP_group(pappas_only=False)

D = count_matrix(groups_VSP, raw_feature.shape[0])
clusters_VSP = spectralClusteringDBSCAN(D, threshold=0,eps=0.001)

print(raw_feature.shape)
print(label_material.shape)

INFO:visiprog.data:After removing non zero entries, N is 3708
INFO:visiprog.data:Number of disconnected elements is 161

(5245, 82)
(5245,)


In [3]:
# print(clusters_VSP)

In [4]:
label_visiprog = get_VSP_label(groups_VSP, raw_feature.shape[0])
label_VSP_clusters = get_VSP_label(clusters_VSP, raw_feature.shape[0])

In [6]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)

In [13]:
list_img = read_img_list()

for n_cluster in range(1, 11):

    accuracy_STSIMM = []
    accuracy_mat = []
    accuracy_vsp_original = []
    accuracy_vsp = []

    for i in range(10):
        X_train, X_test, X_STSIMM_train, X_STSIMM_test, Y_mat_train, Y_mat_test, Y_VSP_train, Y_VSP_test = \
        train_test_split(raw_feature, X_STSIMM, label_material, label_visiprog, \
                         test_size=0.2, stratify=label_material)

        res = kmean_subclass(X_STSIMM_train, Y_mat_train, X_STSIMM_test, Y_mat_test, n_cluster)
        accuracy_STSIMM.append(res['accuracy'])

        model = LFDA()
        try:
            model.fit(X_train, Y_mat_train)
            X_mat_train = model.transform(X_train)
            X_mat_test = model.transform(X_test)
            res = kmean_subclass(X_mat_train, Y_mat_train, X_mat_test, Y_mat_test, n_cluster)
            accuracy_mat.append(res['accuracy'])
        except Exception as e:
            acuracy_mat.append(np.nan)

        model_vsp_original = LFDA()
        try:

            idx_outlier = np.where(Y_VSP_train == 0)[0]
            X_train_without_outlier = X_train[~idx_outlier,:]
            Y_VSP_train_without_outlier = Y_VSP_train[~idx_outlier]
            model_vsp_original.fit(X_train_without_outlier, Y_VSP_train_without_outlier)

            X_vsp_original_train = model_vsp_original.transform(X_train)
            X_vsp_original_test = model_vsp_original.transform(X_test)
            res_vsp_original = kmean_subclass(X_vsp_original_train, Y_mat_train, \
                                              X_vsp_original_test, Y_mat_test, n_cluster)
            print(res_vsp_original['accuracy'])
            accuracy_vsp_original.append(res_vsp_original['accuracy'])
        except Exception as e:
            print(e)
            accuracy_vsp_original.append(np.nan)


        model_vsp = LFDA_VISIPROG()
        try:
            model_vsp.fit(X_train, Y_VSP_train)
            X_vsp_test = model_vsp.transform(X_test)

            # remove trained data which are not labeled by VSP
            indx = np.where(Y_VSP_train != 0)[0]
            X_vsp_train = model_vsp.transform(X_train[indx])
            res_vsp = kmean_subclass(X_vsp_train, Y_mat_train[indx], X_vsp_test, Y_mat_test, n_cluster)
            accuracy_vsp.append(res_vsp['accuracy'])
        except Exception as e:
            accuracy_vsp.append(np.nan)


    df = pd.DataFrame({'STSIM-M':accuracy_STSIMM,'visiprog':accuracy_vsp, \
                       'visiprog_original': accuracy_vsp_original, 'material':accuracy_mat}, \
                      columns=['STSIM-M','visiprog_original','visiprog','material'])
    df.loc['average'] = df.mean()
    display(df)


    df.to_csv('thrasos/within_fromgroup_k{}.csv'.format(n_cluster))



INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[38 45 46 ...,  8 29 20]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[38 45 46 ...,  8 29 20]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[38 45 46 ...,  8 29 20]
INFO:visiprog.evaluate:Running on test set
0.741658722593
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[38 46 40 ...,  8 29 20]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[14 12 10 ..., 55 13  9]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[14 12 10 ..., 55 13  9]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[14 12 10 ..., 55 13  9]
INFO:visiprog.evaluate:Running on test set
0.728312678742
INFO:visiprog.evaluate:KMeans clustering
INFO:

,STSIM-M,visiprog_original,visiprog,material
0,0.816015,0.741659,0.714013,0.804576
1,0.824595,0.728313,0.739752,0.836988
2,0.818875,0.731173,0.715920,0.839847
3,0.819828,0.707340,0.739752,0.843661
4,0.822688,0.772164,0.730219,0.829361
5,0.814109,0.693994,0.719733,0.805529
6,0.834128,0.769304,0.724500,0.797903
7,0.802669,0.740705,0.741659,0.825548
8,0.816015,0.665396,0.707340,0.805529
9,0.810296,0.760724,0.721640,0.854147


INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[25 18 24 ...,  2 41  1]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[25 18 24 ...,  2 41  1]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[25 18 24 ...,  2 41  1]
INFO:visiprog.evaluate:Running on test set
0.808388941849
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[25 18 53 ..., 18 47  2]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[17 53 27 ...,  5 35 37]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[17 53 27 ...,  5 35 37]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[17 53 27 ...,  5 35 37]
INFO:visiprog.evaluate:Running on test set
0.810295519542
INFO:visiprog.evaluate:KMeans clustering
INFO:

,STSIM-M,visiprog_original,visiprog,material
0,0.846520,0.808389,0.868446,0.912297
1,0.836034,0.810296,0.848427,0.908484
2,0.857007,0.784557,0.859867,0.905624
3,0.854147,0.833174,0.855100,0.917064
4,0.850334,0.865586,0.844614,0.897998
5,0.857960,0.843661,0.869399,0.899905
6,0.845567,0.815062,0.857007,0.906578
7,0.840801,0.812202,0.842707,0.901811
8,0.840801,0.826501,0.857007,0.920877
9,0.862726,0.850334,0.873213,0.920877


INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[44 42 37 ..., 29 19 30]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[44 42 37 ..., 29 19 30]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[44 42 37 ..., 29 19 30]
INFO:visiprog.evaluate:Running on test set
0.892278360343
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[44 42 37 ..., 29 19 30]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[13 41  8 ..., 20  2 16]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[13 41  8 ..., 20  2 16]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[13 41  8 ..., 20  2 16]
INFO:visiprog.evaluate:Running on test set
0.887511916111
INFO:visiprog.evaluate:KMeans clustering
INFO:

...

,STSIM-M,visiprog_original,visiprog,material
0,0.896092,0.940896,0.947569,0.976168
1,0.897998,0.939943,0.947569,0.971401
2,0.901811,0.952336,0.950429,0.972355
3,0.881792,0.934223,0.943756,0.965682
4,0.860820,0.938990,0.954242,0.964728
5,0.875119,0.908484,0.962822,0.972355
6,0.891325,0.943756,0.958055,0.968541
7,0.891325,0.920877,0.940896,0.957102
8,0.877026,0.913251,0.938036,0.963775
9,0.883699,0.936130,0.972355,0.969495


INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[23 45 15 ..., 34 23 32]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[23 45 15 ..., 34 23 32]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[23 45 15 ..., 34 23 32]
INFO:visiprog.evaluate:Running on test set
0.919923736892
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[23 55 44 ..., 17 34 32]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[20  2 34 ..., 39 49 24]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[20  2 34 ..., 39 49 24]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[20  2 34 ..., 39 49 24]
INFO:visiprog.evaluate:Running on test set
0.949475691134
INFO:visiprog.evaluate:KMeans clustering
INFO:

,STSIM-M,visiprog_original,visiprog,material
0,0.884652,0.919924,0.960915,0.970448
1,0.882745,0.949476,0.949476,0.975214
2,0.888465,0.936130,0.958055,0.965682
3,0.896092,0.938036,0.950429,0.964728
4,0.879886,0.915157,0.958055,0.970448
5,0.890372,0.950429,0.961868,0.964728
6,0.904671,0.934223,0.948522,0.970448
7,0.897998,0.941849,0.957102,0.972355
8,0.887512,0.920877,0.948522,0.969495
9,0.882745,0.944709,0.957102,0.970448


INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[46  6 19 ..., 31 41 13]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[46  6 19 ..., 31 41 13]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[46  6 19 ..., 31 41 13]
INFO:visiprog.evaluate:Running on test set
0.931363203051
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[46  6 19 ..., 51  9 13]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[49 20 39 ...,  3  7 33]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[49 20 39 ...,  3  7 33]
INFO:visiprog.evaluate:Running on test set
INFO:visiprog.evaluate:KMeans clustering
INFO:visiprog.evaluate:[49 20 39 ...,  3  7 33]
INFO:visiprog.evaluate:Running on test set
0.938036224976
INFO:visiprog.evaluate:KMeans clustering
INFO:

,STSIM-M,visiprog_original,visiprog,material
0,0.868446,0.931363,0.953289,0.980934
1,0.886559,0.938036,0.958055,0.971401
2,0.891325,0.925643,0.951382,0.968541
3,0.873213,0.935176,0.958055,0.967588
4,0.898951,0.943756,0.968541,0.980934
5,0.886559,0.935176,0.946616,0.971401
6,0.891325,0.930410,0.954242,0.973308
7,0.895138,0.952336,0.948522,0.959009
8,0.882745,0.928503,0.955195,0.969495
9,0.885605,0.941849,0.956149,0.971401
